In [16]:
# import dependencies
import os
import io
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone
import pandas as pd
import FoodTools

In [ ]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [ ]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [ ]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [ ]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)
print('Length: ', len(meal_type))

In [ ]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print('Length: ', len(food_list))

In [ ]:
# In case of dupliate food name, the list should be converted to a set
# sort
food_list = sorted(list(set(food_list)))
print(food_list)
print('Sorted length: ', len(food_list))

In [ ]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
# pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
# pref_4 = [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
# pref_5 = [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
pref = []
for i in food_list:
    print(i)
    preference = int(input())
    if preference != 0 and preference != 1:
        print('Incrorrect input')
        break
    pref.append(preference)
print(pref)
print('Preference length: ', len(pref))

In [ ]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
print('Current US/Pacific time: ', us_time)

In [ ]:
# write the food list to a csv file
path = '/Users/7w0r4ng3s/Documents/menu_scraping/data/{}.csv'.format(us_time)
print('Current path: ', path)
path_2 = '/Users/7w0r4ng3s/Documents/menu_scraping/data/'

In [ ]:
def write_data():
    with open(path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref):
            writer.writerow([key, val])
    print('write_csv: COMPLETED')
    # add index and column names
    df = pd.read_csv(path, names=['food', 'pref'])
    df.index.names = ['index']
    df.to_csv(path)
    print('add_column_name: COMPLETED')
    
write_data()

In [ ]:
# def merge_data():
#     # TODO: Figure out a way to get rid of the unnamed: 0 column
#     # TODO: Modify merge_data() so that new data can be append to data.csv
#     files = [f for f in os.listdir('.') if os.path.isfile(f)]

#     merged = []

#     for f in files:
#         filename, ext = os.path.splitext(f)
#         if ext == '.csv':
#             read = pd.read_csv(f)
#             merged.append(read)

#     result = pd.concat(merged)
#     result.to_csv('data.csv')
    
# merge_data()

In [ ]:
def append_data():
    df1 = pd.read_csv('Data.csv', index_col='index')
    df1.index.names = ['index']
    df2 = pd.read_csv(path, index_col='index')
    df3 = df1.append(df2).reset_index()
    df3 = df3.drop('index', 1).sort_values('food').reset_index().drop('index', 1)
    df3.index.names = ['index']
    df3.to_csv('Data.csv')
    
append_data()

In [ ]:
FoodTools.search()

In [17]:
FoodTools.never_eat()

['Atomic Cheese Sauce',
 'Bar Indian',
 'Bell Peppers',
 'Black Beans',
 'Bread Bun',
 'Burger Turkey',
 'Cage Free Eggs',
 'Cage Free Omelette Bar',
 'Cheddar Cheese',
 'Condiments',
 'Feta Cheese',
 'Fresh Organic Spinach',
 'Hawaiian Coleslaw',
 'Jalapeno',
 'Mexican Rice',
 'Oatmeal Gluten-Free',
 'Onions',
 'Original Naan',
 'Oven Roasted Chicken Thigh',
 'Pepperoni Pizza',
 'Roasted Corn',
 'Roasted Seasonal Vegetables',
 'Roasted Vegetables',
 'Sauteed Mushrooms',
 'Sliced Olived',
 'Steamed Seasonal Vegetables',
 'Stir Fry Vegetables',
 'Tuscan Vegetable Soup',
 'Vegan Malibu Burger',
 'Vegan Peanut Granola',
 'Vegan Tenders']